In [1]:
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
lens_samples = Table.read('lens_samples.fits')

In [2]:
lens_samples[0]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,bool,bool,bool,int64
1.3196268626551055,264596483383.2295,4.4275,0.6214770781714286,0.7320195457264107,30.232606079738332,-0.9861918644280009,0.7417372980109783,26.9092,26.7538,25.9745,25.8792,236.61722483491087,0.5571690004369176,0.6259450622792214,0.3459937760977172,20.782409392808418,18.83204064877745,18.17475897845127,17.80022039548253,0.7048565112305543,0.631598379841616,1.0664061300899874,0.7688413613416288,1.5908905321477644,i,2.2542780269433975,False,False,False,0


In [36]:
def lens_finder(thetaE, Re_src, psf_fwhm, magnification, snr_arc):
    #seeing test
    # cond1 = thetaE**2 > (Re_src**2 + (psf_fwhm/2.0)**2)
    # cond1 = 2*thetaE**2 >= (2*Re_src)**2+psf_fwhm**2
    cond1 = thetaE**2 > (Re_src**2 + (psf_fwhm)**2)
    
    #resolved counter image
    cond2 = (magnification*Re_src > psf_fwhm)
    
    #arc snr
    cond3 = snr_arc>20.0
    
    #magnification lower bound
    cond4 = magnification > 3
    
    return np.logical_and.reduce([cond1, cond2, cond3, cond4])

In [37]:
from SimCsstLens.SimLensImage.MockSurvey import MockSurvey
survey = MockSurvey(config_path='.', config_file='csst_setting.yaml')
survey.psf_fwhm_dict

{'g': 0.17719007049930974,
 'r': 0.17719007049930974,
 'i': 0.19031526090666598,
 'z': 0.21656564172137857,
 'stack': 0.21656564172137857}

In [38]:
thetaE = lens_samples['thetaE_s0'].data
Re_src = lens_samples['re_s0'].data
mu = lens_samples['mu'].data
psf_fwhm_list = [
    survey.psf_fwhm_dict['g'], 
    survey.psf_fwhm_dict['r'], 
    survey.psf_fwhm_dict['i'], 
    survey.psf_fwhm_dict['z']
]  #[0.177190, 0.177190, 0.190315, 0.21656564172137857]
snr_list = [
    lens_samples['SNR_g'].data,
    lens_samples['SNR_r'].data,
    lens_samples['SNR_i'].data,
    lens_samples['SNR_z'].data,
]

In [39]:
detect_cond_list = [None]*4
for ii in range(4):
    detect_cond_list[ii] = lens_finder(thetaE, Re_src, psf_fwhm_list[ii], mu, snr_list[ii])
    
single_band_bool = np.logical_or.reduce(detect_cond_list)
print('number of single band detection', np.where(single_band_bool)[0].size)

number of single band detection 52438


In [40]:
psf_fwhm_stack =  survey.psf_fwhm_dict['z']
snr_stack = lens_samples['SNR_stack'].data
stack_band_bool = lens_finder(thetaE, Re_src, psf_fwhm_stack, mu, snr_stack)
print('number of stack band detection', np.where(stack_band_bool)[0].size)

number of stack band detection 103673


In [29]:
(99679-24844)/99679

0.7507599394054917

In [35]:
(99679-74068)/99679

0.2569347605814665

In [41]:
(110000-100000)/110000

0.09090909090909091

In [7]:
single_band_bool

array([False, False, False, ..., False, False, False])

In [8]:
s_bool_table = lens_samples['if_obs_single'].data

In [9]:
np.where(single_band_bool != s_bool_table)

(array([ 142114, 3397146]),)

In [10]:
lens_samples[142114]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,bool,bool,bool,int64
1.5168830187419187,268854314487.72906,1.8098,0.9635260849723911,0.2344240855115115,14.744980139832624,-1.187607790172714,-0.09078073060996318,22.9067,22.966,23.0278,22.9565,255.3788709962412,1.65428775000688,0.7294196940551096,0.23385006462450564,18.89954359464662,17.321305003637082,16.757649438636843,16.473135644273807,23.705179535384605,15.331664875479314,11.298925012078316,8.03632949726326,28.18526712460036,g,2.997196691730436,True,False,True,2172


In [13]:
lens_samples[3397146]

thetaE_s0,mass_s0,z_s0,re_s0,q_s0,pa_s0,x_s0,y_s0,mag_g_s0,mag_r_s0,mag_i_s0,mag_z_s0,vdisp_l,re_l,q_l,z_l,mag_g_l,mag_r_l,mag_i_l,mag_z_l,SNR_g,SNR_r,SNR_i,SNR_z,SNR_stack,best_band,mu,if_ring,if_obs_stack,if_obs_single,noise_seed
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,bool,bool,bool,int64
1.9761687166259978,848008919925.5077,1.8294,1.00253332668594,0.8865099478585281,15.701295280725594,0.6004651339756885,1.349010349315035,22.3616,22.574,21.9323,21.5574,322.2821870980947,1.2310607325425604,0.791077543376361,0.43606019269168866,20.094763888955804,18.178977147549617,17.35455863704028,16.86751295531733,34.9295968368236,21.55645122361124,30.699521607997823,28.22698782072543,57.11193472850041,g,2.999731484080643,False,False,True,5273
